COMP 215 - LAB 3 Classes (NEO)
----------------
#### Name:
#### Date:

This lab exercise introduces `class` as a means of organizing related data and functions.

**Building on new concepts from lab 2**:
  * a `record` is a related collection of data, with fields for each data value
  * an `API` is an "Application Programmers Interface" defining how a programmer interacts with a system.
  * *f-string* simplifies string formatting operations

**New Python Concepts**:
  * the `class` keyword allows you define a new data `type`, with a set of operations on that data.
  * a `dataclass` simplifies class definition for classes that primarily encapsulate a data structure.

As usual, the first code cell simply imports all the modules we'll be using...

In [150]:
import datetime, json, requests
from datetime import datetime
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures
import dataclasses

We'll continue working with [Near Earth Object](https://cneos.jpl.nasa.gov/) data
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

Here's a brief review from Lab 2 on how to use it...

### Review: making a query

Here's a query that gets the record for a single NEO that recently passed by.

In [151]:
API_KEY = '4Hw4uU4Fgbo7BBHGoBEtrpkST4XzcKA18Q4PMqeH'  # substitute your API key here

def get_neos(start_date):
    """ Return a list of NEO for the week starting at start_date """
    url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&api_key={API_KEY}'
    # Fetch last week's NEO feed
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    print(f'{data["element_count"]} Near Earth Objects found for week of {start_date}')
    return [neo for dated_records in data['near_earth_objects'].values() for neo in dated_records ]

def get_neo(id):
    """ Return a NEO record for the given id """
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    return json.loads(response.text)

week_start = '2023-01-15'
neos = get_neos(week_start)
assert len(neos) > 0, f'Oh oh!  No NEOs found for {week_start}'
particularneo = get_neo(neos[-1]['id'])  # get the very latest NEO
pprint(particularneo)

117 Near Earth Objects found for week of 2023-01-15
{'absolute_magnitude_h': 24.86,
 'close_approach_data': [{'close_approach_date': '1949-07-04',
                          'close_approach_date_full': '1949-Jul-04 22:19',
                          'epoch_date_close_approach': -646710060000,
                          'miss_distance': {'astronomical': '0.419726307',
                                            'kilometers': '62790161.51016609',
                                            'lunar': '163.273533423',
                                            'miles': '39015997.166588442'},
                          'orbiting_body': 'Earth',
                          'relative_velocity': {'kilometers_per_hour': '73509.6739172087',
                                                'kilometers_per_second': '20.4193538659',
                                                'miles_per_hour': '45676.0526626122'}},
                         {'close_approach_date': '1950-08-07',
                        

## Exercise 1:  Define an Asteroid class

Define a simple Asteroid class with some basic state variables for a single NEO.  Your Asteroid class should define at least 4 "state variables:”

    id
    name
    estimated_diameter (m)
    is_potentially_hazardous (bool)

Operations must include:
* `__init__(self, ...)` method to initialize a new Asteroid object with specific data values
* `__str__(self)`, and `__repr__(self)` methods that return nicely formatted string representations of the object.
  
OR...
use a `@dataclass` and it will supply most of that boilerplate code for you!

Write a little code to test your new class.

In [152]:
class asteroid:
  """A type of asteroid object which is near Earth, as categorized by NASA with an identification-number, a name, a minimum estimated diameter in meters, and a true/false value depending on if it is a potential hazard to the Earth."""
  def __init__(self, id:str, name:str, m_est_diam_m:float, ishazard:bool):
    self.id = id
    self.name = name
    self.m_est_diam_m = m_est_diam_m
    self.ishazard = ishazard

  def __str__(self):
    return f'id: {self.id}, name: {self.name}, minimum estimated diameter in meters: {self.m_est_diam_m}, is potential hazard: {self.ishazard}'

  def __repr__(self):
    return f'asteroid(\'{self.id}\', \'{self.name}\', \'{self.m_est_diam_m}\', \'{self.ishazard}\')'

a1 = asteroid('54339874','(2023 BM1)', 28.3501249023, False)
print(a1)

id: 54339874, name: (2023 BM1), minimum estimated diameter in meters: 28.3501249023, is potential hazard: False


## Exercise 2: Factory method: Asteriod.from_NEO

We want to be able to construct Asteroid objects easily from the record returned from the NEO API.  

Add an "object factory" method to your class...   

    @classmethod
    from_NEO(cls, neo_id):
        ...

that takes the id for a single NEO, fetches the NEO record from API, constructs and returns an Asteroid object representing that NEO.
This kind of method is called a “Factory” because it constructs an object from raw materials.

Write a little code to test your new class.

In [153]:
#here, due to putting class and classmethod in their own code blocks causing total separation of the two leading to an error when running all codeblocks, I repasted the asteroid class definition into this codeblock. You'll keep seeing repastes like these in future answer codeblocks
class asteroid:
  """A type of asteroid object which is near Earth's orbit, as categorized by NASA with an identification-number, a name, a minimum estimated diameter in meters, and a true/false value depending on if it is a potential hazard to the Earth."""
  def __init__(self, id:str, name:str, m_est_diam_m:float, ishazard:bool):
    self.id = id
    self.name = name
    self.m_est_diam_m = m_est_diam_m
    self.ishazard = ishazard

  def __str__(self):
    return f'id: {self.id}, name: {self.name}, minimum estimated diameter in meters: {self.m_est_diam_m}, is potential hazard: {self.ishazard}'

  def __repr__(self):
    return (self.id, self.name, self.m_est_diam_m, self.ishazard)

  def __call__(self):
    return self

  @classmethod
  def identified_NEO(cls, neo_id):
    got_neo = get_neo(neo_id)
    id = got_neo['id']
    name = got_neo['name']
    m_est_diam_m = got_neo['estimated_diameter']['meters']['estimated_diameter_min']
    ishazard = got_neo['is_potentially_hazardous_asteroid']
    givenNEO = cls(id, name, m_est_diam_m, ishazard)
    return givenNEO

fetched = asteroid.identified_NEO('54339874')
print(fetched)

id: 54339874, name: (2023 BM1), minimum estimated diameter in meters: 28.3501249023, is potential hazard: False


## Exercise 3: Define a CloseApproach class

Each NEO record comes with a list of `close_approach_data`, where each record in this list represents a single “close approach” to another orbiting body.
* Develop a class named `CloseApproach` to represent a single close approach record.
* State variables are

        neo (refrence to related NEO object)
        orbiting body (str)
        approach date (datetime object!)
        miss distance (float - choose units, document it, and be consistent!)
        relative velocity (ditto)
  
Define a "Factory" class method to construct a `CloseApproach` object from one close approach data record (a dictionary object).   
This method takes a `neo` object as input for the NEO to which the close approach data belongs.
Remember to parse the date/time string into a datetime object.

In [154]:
def closestdate(datelist, start_date): #will need this for the classmethod. I learned this by referencing https://stackoverflow.com/a/32237949
  """
  A helper-function for the CAOfactory class method that:
    1. Takes a list of dates from one close-approach-data record and the day used as the start_date in the get_neos function, with every date represented as a string with YYYY-MM-DD formatting
    2. Converts every date in the list and the start_date from string-form to datetime-object form
    3. Sorts the datetime-object list by least to most distant in time from the start_date
    4. Grabs the datetime-object that has the smallest difference between itself and the start_date, and then converts it back into string form to return
  """
  dateobjlist = [datetime.strptime(entry['close_approach_date'], '%Y-%m-%d') for entry in datelist]
  startdateobj = datetime.strptime(start_date, '%Y-%m-%d')
  closestdateobj = min(dateobjlist, key=lambda date: abs(date - startdateobj))
  closestday = closestdateobj.strftime('%Y-%m-%d')
  return closestday

class CloseApproach:
  """A type of data catalog for a single close-approach event of one asteroid, containing the following:
      -The data container representing the near-Earth-object itself
      -The orbiting body which the near-Earth-object was orientated to
      -The datetime-object representing the date of when the close-approach event happened
      -The minimum estimated miss-distance between the near-Earth-object and Earth, measured in kilometers
      -The relative-velocity of the near-Earth-object, in terms of displacement over time measured in kilometers-per-hour
  """
  def __init__(self, neo:object, orbitingbody:str, approachdate:object, missdist_km:float, relvel_kph:float):
    self.neo = neo
    self.orbitingbody = orbitingbody
    self.approachdate = approachdate
    self.missdist_km = missdist_km
    self.relvel_kph = relvel_kph

  def __str__(self):
    return f'neo:{self.neo}, orbitingbody:{self.orbitingbody}, approachdate:{self.approachdate}, missdist_km:{self.missdist_km}, relvel_kph:{self.relvel_kph}'

  @classmethod
  def CAOfactory(cls, neo):
    NEOtuple = asteroid.__repr__(neo)
    neo_id = NEOtuple[0]
    entire_neo = get_neo(neo_id)
    entire_CAD_list = entire_neo['close_approach_data']
    neo_event = closestdate(entire_CAD_list, week_start) #it was unclear to me where I was supposed to get a close approach date from, so I looked for the nearest day to the day of week_start
    neo_event_entry = [entry for entry in entire_CAD_list if entry['close_approach_date'] == neo_event]
    orbitingbody = neo_event_entry[0]['orbiting_body']
    approachdate = neo_event
    missdist_km = float(neo_event_entry[0]['miss_distance']['kilometers'])
    relvel_kph = float(neo_event_entry[0]['relative_velocity']['kilometers_per_hour'])
    givenCAO = cls(neo.__repr__(), orbitingbody, approachdate, missdist_km, relvel_kph)
    return givenCAO

taken = CloseApproach.CAOfactory(fetched)
print(taken)

neo:('54339874', '(2023 BM1)', 28.3501249023, False), orbitingbody:Earth, approachdate:2023-01-18, missdist_km:12324076.417537022, relvel_kph:42918.8831905878



## Exercise 4: Add list of CloseApproach objects to the Asteroid

Every `Asteroid` should have a list of “close approaches”.
But there is a catch-22 here because we need the `Asteroid` to construct the `CloseApproach` objects.
Add an instance variable to your Asteroid class with a default value of an empty list:

    ...
    close_approaches:list = []
    ...
      
In `from_NEO` factory, use a list comprehension to build the list of `CloseApproach` objects for the Asteroid instance, and then set the instance's `close_approaches` variable before returning it.  
Setting the value of an object's instance variables from outside the class is generally frowned upon - this is why we make the factory a method of the class itself!

Now add a method to `Asteroid` to return the `nearest_miss` `CloseApproach` object for the asteroic:

    def nearest_miss(self):
        ...

Extend your test code to demonstrate these new features.

In [155]:
class asteroid:
  """
  A type of asteroid object which is near Earth's orbit, as categorized by NASA with:
  -an identification-number
  -a name
  -a minimum estimated diameter in meters
  -a true/false value depending on if it is a potential hazard to the Earth.
  -a list of close approach events
  """
  def __init__(self, id:str, name:str, m_est_diam_m:float, ishazard:bool, close_approaches:list = []):
    self.id = id
    self.name = name
    self.m_est_diam_m = m_est_diam_m
    self.ishazard = ishazard
    self.close_approaches = close_approaches

  def __str__(self):
    return f'id: {self.id}, name: {self.name}, minimum estimated diameter in meters: {self.m_est_diam_m}, is potential hazard: {self.ishazard}, close_approaches:{self.close_approaches}'

  def __repr__(self):
    return (self.id, self.name, self.m_est_diam_m, self.ishazard, self.close_approaches)

  def __call__(self):
    return self

  @classmethod
  def identified_NEO(cls, neo_id):
    got_neo = get_neo(neo_id)
    id = got_neo['id']
    name = got_neo['name']
    m_est_diam_m = got_neo['estimated_diameter']['meters']['estimated_diameter_min']
    ishazard = got_neo['is_potentially_hazardous_asteroid']
    close_approaches = [entry for entry in got_neo['close_approach_data']]
    givenNEO = cls(id, name, m_est_diam_m, ishazard, close_approaches)
    return givenNEO

  def nearest_miss(self):
    numlist = [float(entry['miss_distance']['kilometers']) for entry in self.close_approaches]
    smallest_dist = min(numlist)
    smallest_dist_entry = [entry for entry in self.close_approaches if entry['miss_distance']['kilometers'] == str(smallest_dist)]
    neo = asteroid.identified_NEO(self.id)
    orbitingbody =  smallest_dist_entry[0]['orbiting_body']
    approachdate = smallest_dist_entry[0]['close_approach_date'] #this means that nearest_miss is a different factory from CAOfactory, because there is only one date to choose from (the day of the nearest miss)
    missdist_km = float(smallest_dist_entry[0]['miss_distance']['kilometers'])
    relvel_kph = float(smallest_dist_entry[0]['relative_velocity']['kilometers_per_hour'])
    return CloseApproach(neo.__repr__(), orbitingbody, approachdate, missdist_km, relvel_kph)

def closestdate(datelist, start_date):
  """
  A helper-function for the CAOfactory class method that:
    1. Takes a list of dates from one close-approach-data record and the day used as the start_date in the get_neos function, with every date represented as a string with YYYY-MM-DD formatting
    2. Converts every date in the list and the start_date from string-form to datetime-object form
    3. Sorts the datetime-object list by least to most distant in time from the start_date
    4. Grabs the datetime-object that has the smallest difference between itself and the start_date, and then converts it back into string form to return
  """
  dateobjlist = [datetime.strptime(entry['close_approach_date'], '%Y-%m-%d') for entry in datelist]
  startdateobj = datetime.strptime(start_date, '%Y-%m-%d')
  closestdateobj = min(dateobjlist, key=lambda date: abs(date - startdateobj))
  closestday = closestdateobj.strftime('%Y-%m-%d')
  return closestday

class CloseApproach:
  """A type of data catalog for a single close-approach event of one asteroid, containing the following:
      -The data container representing the near-Earth-object itself
      -The orbiting body which the near-Earth-object was orientated to
      -The datetime-object representing the date of when the close-approach event happened
      -The minimum estimated miss-distance between the near-Earth-object and Earth, measured in kilometers
      -The relative-velocity of the near-Earth-object, in terms of displacement over time measured in kilometers-per-hour
  """
  def __init__(self, neo:object, orbitingbody:str, approachdate:object, missdist_km:float, relvel_kph:float):
    self.neo = neo
    self.orbitingbody = orbitingbody
    self.approachdate = approachdate
    self.missdist_km = missdist_km
    self.relvel_kph = relvel_kph

  def __str__(self):
    return f'neo:{self.neo}, orbitingbody:{self.orbitingbody}, approachdate:{self.approachdate}, missdist_km:{self.missdist_km}, relvel_kph:{self.relvel_kph}'

  def __repr__(self):
    return (self.neo, self.orbitingbody, self.approachdate, self.missdist_km, self.relvel_kph)

  @classmethod
  def CAOfactory(cls, neo):
    NEOtuple = asteroid.__repr__(neo)
    neo_id = NEOtuple[0]
    entire_neo = get_neo(neo_id)
    entire_CAD_list = entire_neo['close_approach_data']
    neo_event = closestdate(entire_CAD_list, week_start)
    neo_event_entry = [entry for entry in entire_CAD_list if entry['close_approach_date'] == neo_event]
    orbitingbody = neo_event_entry[0]['orbiting_body']
    approachdate = neo_event
    missdist_km = float(neo_event_entry[0]['miss_distance']['kilometers'])
    relvel_kph = float(neo_event_entry[0]['relative_velocity']['kilometers_per_hour'])
    givenCAO = cls(neo.__repr__(), orbitingbody, approachdate, missdist_km, relvel_kph)
    return givenCAO

fetched = asteroid.identified_NEO('54339874')
print(fetched)
nearest_miss_obj = asteroid.nearest_miss(fetched)
print(nearest_miss_obj.__repr__()[1:])

id: 54339874, name: (2023 BM1), minimum estimated diameter in meters: 28.3501249023, is potential hazard: False, close_approaches:[{'close_approach_date': '1949-07-04', 'close_approach_date_full': '1949-Jul-04 22:19', 'epoch_date_close_approach': -646710060000, 'relative_velocity': {'kilometers_per_second': '20.4193538659', 'kilometers_per_hour': '73509.6739172087', 'miles_per_hour': '45676.0526626122'}, 'miss_distance': {'astronomical': '0.419726307', 'lunar': '163.273533423', 'kilometers': '62790161.51016609', 'miles': '39015997.166588442'}, 'orbiting_body': 'Earth'}, {'close_approach_date': '1950-08-07', 'close_approach_date_full': '1950-Aug-07 12:47', 'epoch_date_close_approach': -612270780000, 'relative_velocity': {'kilometers_per_second': '11.1844693277', 'kilometers_per_hour': '40264.0895797362', 'miles_per_hour': '25018.5394391422'}, 'miss_distance': {'astronomical': '0.2356866436', 'lunar': '91.6821043604', 'kilometers': '35258219.870009132', 'miles': '21908441.9192727416'}, '

## Challenge - Take your skills to the next level...
### Exercise 5: add one additional analysis

 With these data structures in place, we can now start answering all kinds of interesting questions about a single Asteroid or a set of Asteroids.  
Here’s a couple ideas to try:

* add a method to the Asteroid class, `closest_earth_approach`, that returns the CloseApproach object that represents the closest approach the Asteroid makes to Earth.

* write a **function** named `most_dangerous_approach`, that takes a date range and returns a single “potentially hazardous” Asteroid object that makes the closest approach to Earth in within that range.  Your algorithm will ultimately need to:
    * grab the list of NEO’s for the given date range;
    * use a list comprehension to build the list of Asteroid objects for the NEO’s returned
    * use a list comprehension to filter  potentially hazardous Asteroids only;
    * use a list comprehension to map each Asteroid to its  closest_earth_approach
    * apply Python’s min function to identify the Asteroid with the closest_earth_approach

You may want to decompose some of these steps into smaller functions.
* add a method to the Asteroid class, estimated_mass, that computes an estimate of the Asteroid’s mass based on its diameter.  This is a model – state your assumptions.
* add a method to the CloseApproach class, impact_force,  that estimates the force of impact if the Asteroid hit the orbiting object.  Again, this is a model, state your assumptions.

In [156]:
# Ex. 5 (challenge) your code here